# Model Monitoring

## Initial set up (and pre-requisites)
1. Make sure you have the `mlrun-api` as a Grafana data source configured in your Grafana instance. If not configured,
otherwise add it by:
   1. Open your grafana instance
   2. Navigate to `Configuration -> Data Sources`
   3. Press `Add data source` and configure the following parameters
    ```
    Name: mlrun-api
    URL: http://mlrun-api:8080/api/grafana-proxy/model-endpoints
    Access: Server (default)

    ## Add a custom header of:
    X-V3io-Session-Key: <YOUR ACCESS KEY>
    ```
   4. Press `Save & Test` to make sure it works, a confirmation message should appear when this button is pressed

2. Import the available [dashboards](./dashboards) in your Grafana instance
3. To allow the system to utilize drift measurement, make sure you supply the train set when logging the model on the
   training step

    ```python
    # Log model
    context.log_model(
        "model",
        body=dumps(model),
        artifact_path=context.artifact_subpath("models"),
        extra_data=eval_metrics,
        model_file="model.pkl",
        metrics=context.results,
        training_set=<TRAIN_SET>,  # <-
        label_cols=<LABEL_COLS>,  # <-
        labels={"class": "sklearn.linear_model.LogisticRegression"}
    )
    ```
4. When serving the model, make sure that the Nuclio function is deployed with tracking enabled by applying
   `fn.set_tracking()` on the serving function

## Configuration
The stream processing portion of the model monitoring, can be deployed under multiple configuration options. The
available configurations can be found under `stream.Config`. Once configured it should be supplied as environment
parameters to the Nuclio function by setting `fn.set_envs`

```python
project: str                        # project name
sample_window: int                  # The sampling window for the data that flows into the TSDB and the KV
tsdb_batching_max_events: int       # The max amount of event to batch before writing the batch to tsdb
tsdb_batching_timeout_secs: int     # The max amount of seconds a given batch can be gathered before being emitted
parquet_batching_max_events: int    # The max amount of event to batch before writing the batch to parquet
parquet_batching_timeout_secs: int  # The max amount of seconds, a given batch can be gathered before being written to parquet
aggregate_count_windows: List[str]  # List of window sizes for predictions count
aggregate_count_period: str         # Period of predictions count windows
aggregate_avg_windows: List[str]    # List of window sizes for average latency
aggregate_avg_period: str           # Period of average latency windows
v3io_access_key: str                # V3IO Access key, if not set will be taken from environment
v3io_framesd: str                   # V3IO framesd URL, if not set will be taken from environment
```

In [ ]:
# Set project name
project = ""

## Deploy Stream Processing

In [ ]:
from mlrun import import_function, mount_v3io
from mlrun.runtimes import RemoteRuntime

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.metadata.project = project
fn.apply(mount_v3io())
fn.deploy()

## Deploy Batch Processing

In [ ]:
from mlrun import import_function, mount_v3io
from mlrun.runtimes import KubejobRuntime

fn: KubejobRuntime = import_function("hub://model_monitoring_batch")
fn.apply(mount_v3io())
fn.run(name='model-monitoring-batch', schedule="0 */1 * * *", params={"project": project})